In [1]:
import findspark
findspark.init()

# Initialize SparkSession

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession\
    .builder\
    .appName("StructuredNetworkWordCount")\
    .getOrCreate()

# Create Streaming Dataframe 


In [3]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host","localhost")\
    .option("port",9999)\
    .load()

words = lines.select(
    explode(
        split(lines.value, " ")
    ).alias("word")
)

wordCounts = words.groupBy("word").count()

In [4]:

# Start running the query that prints the running counts to the console
query = wordCounts.writeStream.outputMode("complete").format("console").option("checkpointLocation", "path/to/checkpoint/dir").start()

# Await termination of the query
query.awaitTermination()


StreamingQueryException: [STREAM_FAILED] Query [id = 5cc28469-187b-42f1-beb9-db424f356e4e, runId = 4fc24681-d641-4300-9523-9dbf3ecd57b5] terminated with exception: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z

## Todo: **Try JAVA 8**

In [5]:
import os
checkpoint_dir = "C:/tmp/hive"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)


In [6]:
spark.sparkContext.setLogLevel("DEBUG")
